In [1]:
import sys
sys.path.append('/Users/mnannan/dev/supelec/ngsa/assignments/Assignement 2 ')

from main import pipeline

from core.io.loader import *

from features.features_extractor import FeaturesExtractor

from features.processing import features_processing
from models.cross_validation import *

from plot.features_importances import plot_features_importances

%matplotlib inline

features_extractor = FeaturesExtractor(pipeline)

train = get_data_with_node_information('train', data_dir='../data')

train = train.sample(frac=1.0,random_state=123)

test = train[492409:]
train = train[:492409]

train.source_id.nunique()

train = features_extractor.fit_transform(train)

test = features_extractor.transform(test)

selected_columns = [
    'title_overlap',
    'publication_date_diff',
    'common_authors',
    'same_journal',
    'authors_collaboration',
    'collaboration_min_distance',
    'collaboration_mean_distance',
    'writer_in_target_paper',
    'writer_collaborators',
    'writer_collaboration_min_distance',
    'writer_collaboration_mean_distance',
    'source_target_common_neighbors',
    'source_paper_citations',
    'target_paper_citations',
    'source_number_of_papers_cited',
    'target_number_of_papers_cited',
]

x_train = features_processing(train[selected_columns]).values
x_test = features_processing(test[selected_columns]).values
y_train = train.category.values
y_test = test.category.values


from sklearn.ensemble import RandomForestClassifier
seed = 123
model_args = {
    'n_estimators': 100,
    'max_depth': 7,
    'random_state': seed
}

y_pred, f1_list = cross_validation(RandomForestClassifier, model_args, x_train, y_train)

model = RandomForestClassifier(**model_args)
model.fit(x_train, y_train)

plot_features_importances(model.feature_importances_, selected_columns)

y_pred = model.predict(x_test)

f1_score(y_test, y_pred)